# The Context-Aware MAS Implementation

Copyright 2025, Denis Rothman

Notebook này triển khai một kiến trúc **Multi-Agent System (MAS)** (Hệ thống đa tác nhân) phục vụ mục đích giáo dục, sử dụng phương pháp **RAG** thông qua **Pinecone** với giao thức **MCP**.

Notebook này thực thi lớp xử lý (**execution layer**) của hệ thống **Context-Aware** (nhận biết ngữ cảnh) của chúng ta. Chúng ta sẽ xây dựng một **Multi-Agent System (MAS)** hoàn chỉnh, nơi các **specialized agents** (tác nhân chuyên biệt) phối hợp để hoàn thành một mục tiêu cấp cao, bằng cách khai thác dữ liệu đã được **ingest** vào **Pinecone** trước đó. Kiến trúc cốt lõi được thiết kế để tách biệt rõ ràng giữa **procedural instructions** (hướng dẫn về quy trình - cái "làm như thế nào") và **factual data** (dữ liệu thực tế - cái "gì"), từ đó cho phép tạo nội dung một cách linh hoạt và có kiểm soát.

Dưới đây là chi tiết kế hoạch thực hiện:

### 1. Định nghĩa Agent

Chúng ta sẽ lập trình ba **specialized agents** đóng vai trò là hạt nhân của hệ thống:

* **The Context Librarian:** Thực hiện **procedural RAG** để truy xuất các "**Semantic Blueprints**" (bản thiết kế ngữ nghĩa) mang tính phong cách.
* **The Researcher:** Sử dụng **factual RAG** để truy vấn và tổng hợp tri thức về một chủ đề nhất định.
* **The Writer:** Kết hợp một cách thông minh giữa **blueprint** và kết quả **research** để tạo ra kết quả đầu cuối (**final output**).

### 2. The Orchestrator (Điều phối viên)

Agent này đóng vai trò quản lý. Nó sử dụng một **LLM** để phân tích mục tiêu của người dùng, chia nhỏ mục tiêu đó thành các truy vấn về **intent** (ý định) và **topic** (chủ đề) riêng biệt cho các agent khác.

### 3. Agent Communication (Giao tiếp giữa các Agent)

Một **Message Communication Protocol (MCP)** đơn giản được định nghĩa để đảm bảo các agent tương tác với nhau theo cách có cấu trúc và có thể truy vết (**traceable**).

### 4. End-to-End Execution (Thực thi đầu cuối)

Chúng ta sẽ chạy một vài ví dụ để chứng minh cách hệ thống **MAS** này có thể tạo ra các đầu ra độc nhất cho nhiều chủ đề khác nhau bằng cách truy xuất động các ngữ cảnh (**context**) và tri thức (**knowledge**) chính xác.


In [1]:
# Imports and API Key Setup
# We will use the OpenAI library to interact with the LLM and Google Colab's
# secret manager to securely access your API key.

import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# The client will automatically read the OPENAI_API_KEY from your environment.
client = OpenAI()
print("OpenAI client initialized.")

OpenAI client initialized.


In [2]:
# Configuration
EMBEDDING_MODEL = "text-embedding-3-small"
EMBEDDING_DIM = 1536 # Dimension for text-embedding-3-small
GENERATION_MODEL = "gpt-5.1"

In [3]:
# Imports for this notebook
import json
import time
from tqdm.auto import tqdm
import tiktoken
from pinecone import Pinecone, ServerlessSpec
from tenacity import retry, stop_after_attempt, wait_random_exponential
# general imports required in the notebooks of this book
import re
import textwrap
from IPython.display import display, Markdown
import copy

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')


/home/dino/Documents/learning-2026/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2.Initialize Clients

In [4]:
# 2.Initialize Clients
# --- Initialize Clients (assuming this is already done) ---

# --- Initialize Pinecone Client ---
pc = Pinecone(api_key=PINECONE_API_KEY)

# --- Define Index and Namespaces (assuming this is already done) ---
INDEX_NAME = 'genai-mas-mcp-ch3'
NAMESPACE_KNOWLEDGE = "KnowledgeStore"
NAMESPACE_CONTEXT = "ContextLibrary"
spec = ServerlessSpec(cloud='aws', region='us-east-1')

# Check if index exists
if INDEX_NAME not in pc.list_indexes().names():
    print(f"Index '{INDEX_NAME}' not found. Creating new serverless index...")
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBEDDING_DIM, # Make sure EMBEDDING_DIM is defined
        metric='cosine',
        spec=spec
    )
    # Wait for index to be ready
    while not pc.describe_index(INDEX_NAME).status['ready']:
        print("Waiting for index to be ready...")
        time.sleep(1)
    print("Index created successfully. It is new and empty.")
else:
    # This block runs ONLY if the index already existed.
    print(f"Index '{INDEX_NAME}' already exists.")

    # Connect to the index to perform operations
    index = pc.Index(INDEX_NAME)

# Connect to the index for subsequent operations
index = pc.Index(INDEX_NAME)


Index 'genai-mas-mcp-ch3' already exists.


# 3.Helper Functions (LLM, Embeddings, and MCP)

In [5]:
#3.Helper Functions (LLM, Embeddings, and MCP)
# -------------------------------------------------------------------------
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def call_llm(system_prompt, user_prompt, temperature=1, json_mode=False):
    """A centralized function to handle all LLM interactions with retries."""
    try:
        response_format = {"type": "json_object"} if json_mode else {"type": "text"}
        response = client.chat.completions.create(
            model=GENERATION_MODEL,
            response_format=response_format,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=temperature
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error calling LLM: {e}")
        return f"LLM Error: {e}"

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def get_embedding(text):
    """Generates embeddings for a single text query with retries."""
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=EMBEDDING_MODEL)
    return response.data[0].embedding

def create_mcp_message(sender, content, metadata=None):
    """Creates a standardized MCP message (Educational Version)."""
    return {
        "protocol_version": "1.1 (RAG-Enhanced)",
        "sender": sender,
        "content": content,
        "metadata": metadata or {}
    }

def display_mcp(message, title="MCP Message"):
    """Helper function to display MCP messages clearly during the trace."""
    print(f"\n--- {title} (Sender: {message['sender']}) ---")
    # Display content snippet or keys if content is complex
    if isinstance(message['content'], dict):
         print(f"Content Keys: {list(message['content'].keys())}")
    else:
        print(f"Content: {textwrap.shorten(str(message['content']), width=100)}")
    # Display metadata keys
    print(f"Metadata Keys: {list(message['metadata'].keys())}")
    print("-" * (len(title) + 25))

def query_pinecone(query_text, namespace, top_k=1):
    """Embeds the query text and searches the specified Pinecone namespace."""
    try:
        query_embedding = get_embedding(query_text)
        response = index.query(
            vector=query_embedding,
            namespace=namespace,
            top_k=top_k,
            include_metadata=True
        )
        return response['matches']
    except Exception as e:
        print(f"Error querying Pinecone (Namespace: {namespace}): {e}")
        return []

print("Helper functions and MCP structure defined.")

Helper functions and MCP structure defined.


In [7]:
#@title 4.Agent Definitions
# -------------------------------------------------------------------------

# === 4.1. Context Librarian Agent (Procedural RAG) ===
def agent_context_librarian(mcp_message):
    """
    Retrieves the appropriate Semantic Blueprint from the Context Library.
    """
    print("\n[Librarian] Activated. Analyzing intent...")
    requested_intent = mcp_message['content']['intent_query']

    # Query Pinecone Context Namespace
    results = query_pinecone(requested_intent, NAMESPACE_CONTEXT, top_k=1)

    if results:
        match = results[0]
        print(f"[Librarian] Found blueprint '{match['id']}' (Score: {match['score']:.2f})")
        # Retrieve the blueprint JSON string stored in metadata
        blueprint_json = match['metadata']['blueprint_json']
        content = {"blueprint": blueprint_json}
    else:
        print("[Librarian] No specific blueprint found. Returning default.")
        # Fallback default
        content = {"blueprint": json.dumps({"instruction": "Generate the content neutrally."})}

    return create_mcp_message("Librarian", content)

# === 4.2. Researcher Agent (Factual RAG) ===
def agent_researcher(mcp_message):
    """
    Retrieves and synthesizes factual information from the Knowledge Base.
    """
    print("\n[Researcher] Activated. Investigating topic...")
    topic = mcp_message['content']['topic_query']

    # Query Pinecone Knowledge Namespace
    results = query_pinecone(topic, NAMESPACE_KNOWLEDGE, top_k=3)

    if not results:
        print("[Researcher] No relevant information found.")
        return create_mcp_message("Researcher", {"facts": "No data found."})

    # Synthesize the findings (Retrieve-and-Synthesize)
    print(f"[Researcher] Found {len(results)} relevant chunks. Synthesizing...")
    source_texts = [match['metadata']['text'] for match in results]

    system_prompt = """Bạn là một chuyên gia AI về tổng hợp nghiên cứu (research synthesis). 
Hãy tổng hợp các văn bản nguồn (source texts) đã cung cấp thành một bản tóm tắt ngắn gọn, trình bày dưới dạng danh sách (bullet-pointed summary) và bám sát chủ đề của người dùng. 
Yêu cầu:
1. Tập trung nghiêm ngặt vào các sự kiện thực tế (facts) có trong nguồn tài liệu. 
2. Tuyệt đối không thêm thông tin bên ngoài (outside information)."""

    user_prompt = f"Topic: {topic}\n\nSources:\n" + "\n\n---\n\n".join(source_texts)

    findings = call_llm(system_prompt, user_prompt)

    return create_mcp_message("Researcher", {"facts": findings})

# === 4.3. Writer Agent (Generation) ===
def agent_writer(mcp_message):
    """
    Combines the factual research with the semantic blueprint to generate the final output.
    """
    print("\n[Writer] Activated. Applying blueprint to facts...")

    facts = mcp_message['content']['facts']
    # The blueprint is passed as a JSON string
    blueprint_json_string = mcp_message['content']['blueprint']

    # The Writer's System Prompt incorporates the dynamically retrieved blueprint
    system_prompt = f"""Bạn là một chuyên gia AI về tạo nội dung (content generation).
    Nhiệm vụ của bạn là tạo nội dung dựa trên các KẾT QUẢ NGHIÊN CỨU (RESEARCH FINDINGS) được cung cấp.
    Quan trọng nhất, bạn PHẢI cấu trúc, định hình phong cách và thiết lập các giới hạn cho kết quả đầu ra dựa trên các quy tắc được định nghĩa trong BẢN THIẾT KẾ NGỮ NGHĨA (SEMANTIC BLUEPRINT) dưới đây.

    --- SEMANTIC BLUEPRINT (JSON) ---
    {blueprint_json_string}
    --- END SEMANTIC BLUEPRINT ---

    Hãy tuân thủ nghiêm ngặt các hướng dẫn, chỉ dẫn phong cách (style guides) và mục tiêu trong blueprint. 
    Blueprint xác định CÁCH bạn viết (HOW); nghiên cứu xác định nội dung bạn viết về CÁI GÌ (WHAT).
    """

    user_prompt = f"""
    --- RESEARCH FINDINGS ---
    {facts}
    --- END RESEARCH FINDINGS ---

    Generate the content now.
    """

    # Generate the final content (slightly higher temperature for potential creativity)
    final_output = call_llm(system_prompt, user_prompt)

    return create_mcp_message("Writer", {"output": final_output})

In [8]:
#@title 5.The Orchestrator
# -------------------------------------------------------------------------

def orchestrator(high_level_goal):
    """
    Manages the workflow of the Context-Aware MAS.
    Analyzes the goal, retrieves context and facts, and coordinates generation.
    """
    print(f"=== [Orchestrator] Starting New Task ===")
    print(f"Goal: {high_level_goal}")

    # Step 0: Analyze Goal (Determine Intent and Topic)
    # We use the LLM to separate the desired style (intent) from the subject matter (topic).
    print("\n[Orchestrator] Analyzing Goal...")
    analysis_system_prompt = """Bạn là một chuyên gia phân tích mục tiêu (goal analyst). 
Hãy phân tích mục tiêu cấp cao (high-level goal) của người dùng và trích xuất hai thành phần sau:

1. 'intent_query': Một cụm từ mô tả tóm tắt phong cách, giọng văn, hoặc định dạng mong muốn, được tối ưu hóa để tìm kiếm trong thư viện ngữ cảnh (context library). 
   (Ví dụ: "suspenseful narrative blueprint", "objective technical explanation structure").

2. 'topic_query': Một cụm từ súc tích tóm tắt chủ đề thực tế hoặc nội dung chuyên môn cần thiết. 
   (Ví dụ: "Juno mission objectives and power", "Apollo 11 landing details").

Yêu cầu: CHỈ phản hồi bằng một đối tượng JSON chứa hai khóa (keys) này."""

    # We request JSON mode for reliable parsing
    analysis_result = call_llm(analysis_system_prompt, high_level_goal, json_mode=True)

    try:
        analysis = json.loads(analysis_result)
        intent_query = analysis['intent_query']
        topic_query = analysis['topic_query']
    except (json.JSONDecodeError, KeyError):
        print(f"[Orchestrator] Error: Could not parse analysis JSON. Raw Analysis: {analysis_result}. Aborting.")
        return

    print(f"Orchestrator: Intent Query: '{intent_query}'")
    print(f"Orchestrator: Topic Query: '{topic_query}'")


    # Step 1: Get the Context Blueprint (Procedural RAG)
    mcp_to_librarian = create_mcp_message(
        sender="Orchestrator",
        content={"intent_query": intent_query}
    )
    # display_mcp(mcp_to_librarian, "Orchestrator -> Librarian")
    mcp_from_librarian = agent_context_librarian(mcp_to_librarian)
    display_mcp(mcp_from_librarian, "Librarian -> Orchestrator")

    context_blueprint = mcp_from_librarian['content'].get('blueprint')
    if not context_blueprint: return

    # Step 2: Get the Factual Knowledge (Factual RAG)
    mcp_to_researcher = create_mcp_message(
        sender="Orchestrator",
        content={"topic_query": topic_query}
    )
    # display_mcp(mcp_to_researcher, "Orchestrator -> Researcher")
    mcp_from_researcher = agent_researcher(mcp_to_researcher)
    display_mcp(mcp_from_researcher, "Researcher -> Orchestrator")

    research_findings = mcp_from_researcher['content'].get('facts')
    if not research_findings: return

    # Step 3: Generate the Final Output
    # Combine the outputs for the Writer Agent
    writer_task = {
        "blueprint": context_blueprint,
        "facts": research_findings
    }

    mcp_to_writer = create_mcp_message(
        sender="Orchestrator",
        content=writer_task
    )
    # display_mcp(mcp_to_writer, "Orchestrator -> Writer")
    mcp_from_writer = agent_writer(mcp_to_writer)
    display_mcp(mcp_from_writer, "Writer -> Orchestrator")

    final_result = mcp_from_writer['content'].get('output')

    print("\n=== [Orchestrator] Task Complete ===")
    return final_result

#  6.Running examples

In [9]:
#@title Example 1: Requesting a specific style (Suspense) for a topic (Apollo 11)
print("********  1: SUSPENSEFUL NARRATIVE **********")
goal_1 = "Viết một phân cảnh ngắn, đầy kịch tính cho một câu chuyện thiếu nhi về sự kiện tàu Apollo 11 đổ bộ lên Mặt Trăng, trong đó làm nổi bật những mối nguy hiểm."
result_1 = orchestrator(goal_1)

print("\n******** FINAL OUTPUT 1 **********\n")
print(result_1)

print("\n\n" + "="*50 + "\n\n")

********  1: SUSPENSEFUL NARRATIVE **********
=== [Orchestrator] Starting New Task ===
Goal: Viết một phân cảnh ngắn, đầy kịch tính cho một câu chuyện thiếu nhi về sự kiện tàu Apollo 11 đổ bộ lên Mặt Trăng, trong đó làm nổi bật những mối nguy hiểm.

[Orchestrator] Analyzing Goal...
Orchestrator: Intent Query: 'dramatic childrens story scene structure'
Orchestrator: Topic Query: 'Apollo 11 moon landing dangers for kids'

[Librarian] Activated. Analyzing intent...
[Librarian] Found blueprint 'blueprint_suspense_narrative' (Score: 0.27)

--- Librarian -> Orchestrator (Sender: Librarian) ---
Content Keys: ['blueprint']
Metadata Keys: []
--------------------------------------------------

[Researcher] Activated. Investigating topic...
[Researcher] Found 3 relevant chunks. Synthesizing...

--- Researcher -> Orchestrator (Sender: Researcher) ---
Content Keys: ['facts']
Metadata Keys: []
---------------------------------------------------

[Writer] Activated. Applying blueprint to facts...

--